In [2]:
# pipeline_gnn.py
import sys
sys.path.append("../")
import numpy as np
import pandas as pd
from pipeline.modules import CambModule, ClassModule, EmceeModule
from pipeline.functions import log_posterior, run_emcee, camb_calc

In [2]:
# -------------------------
# Load reference data
# -------------------------
pk_read = pd.read_csv('../Data/std-camb-data.csv')
pk_std = pk_read['pk'].values

cosmo = [67.4, 0.02237, 0.12, 0.06, 2.10e-9, 0.965]

camb_mod = CambModule(fixed_params={"cosmo_params": cosmo},
                      var_params={"AccuracyBoost": list(np.linspace(1.0, 3.0, 11)),
                                  "lAccuracyBoost": list(np.linspace(1.0, 3.0, 11)),
                                  "lSampleBoost": list(np.linspace(1.0, 3.0, 11))}
                     )

class_mod = ClassModule(fixed_params={"cosmo_params": cosmo},
                        var_params={"tol_background": [1e-6, 1e-8, 1e-10],
                                    "tol_thermo": [1e-4, 1e-5, 1e-6]}
                       )

mean = np.array([0.0, 1.0, 2.0, 3.0, 4.0, 5.0])

# 6x6 covariance matrix (symmetric, positive-definite)
cov = np.array([
    [1.0, 0.2, 0.1, 0.0, 0.0, 0.0],
    [0.2, 1.5, 0.2, 0.1, 0.0, 0.0],
    [0.1, 0.2, 1.2, 0.1, 0.0, 0.0],
    [0.0, 0.1, 0.1, 1.0, 0.1, 0.0],
    [0.0, 0.0, 0.0, 0.1, 0.8, 0.1],
    [0.0, 0.0, 0.0, 0.0, 0.1, 0.9]
])
initial_guess = np.zeros(6)

Input = {"mean": mean,"cov": cov}

std_df = pd.DataFrame({"mu" : mean, "sigma" : np.sqrt(np.diag(cov))})


emcee_mod = EmceeModule(fixed_params={"initial_guess": initial_guess,
                                      "loglike_func": log_posterior,
                                      "like_args": Input},
                        var_params={"n_walkers": [50, 100],
                                    "n_steps": [300, 600]})

In [3]:
%%time
camb_mod.run()

CPU times: user 5.88 s, sys: 21.6 ms, total: 5.9 s
Wall time: 489 ms


{'cosmo_params': [67.4, 0.02237, 0.12, 0.06, 2.1e-09, 0.965],
 'result':              pk
 0    422.115466
 1    446.339226
 2    471.951445
 3    499.031446
 4    527.662951
 ..          ...
 195    0.407094
 196    0.350308
 197    0.301353
 198    0.259163
 199    0.222817
 
 [200 rows x 1 columns],
 'cpu_time': 0.48935532569885254}

In [4]:
%%time
class_mod.run()

CPU times: user 2.13 s, sys: 56.3 ms, total: 2.18 s
Wall time: 232 ms


{'cosmo_params': [67.4, 0.02237, 0.12, 0.06, 2.1e-09, 0.965],
 'result':              pk
 0    422.123872
 1    446.348087
 2    471.960808
 3    499.041294
 4    527.673287
 ..          ...
 195    0.405371
 196    0.348795
 197    0.300011
 198    0.257981
 199    0.221801
 
 [200 rows x 1 columns],
 'cpu_time': 0.23169851303100586}

In [5]:
emcee_mod.run()

{'initial_guess': array([0., 0., 0., 0., 0., 0.]),
 'loglike_func': <function pipeline.functions.log_posterior(theta, like_args)>,
 'like_args': {'mean': array([0., 1., 2., 3., 4., 5.]),
  'cov': array([[1. , 0.2, 0.1, 0. , 0. , 0. ],
         [0.2, 1.5, 0.2, 0.1, 0. , 0. ],
         [0.1, 0.2, 1.2, 0.1, 0. , 0. ],
         [0. , 0.1, 0.1, 1. , 0.1, 0. ],
         [0. , 0. , 0. , 0.1, 0.8, 0.1],
         [0. , 0. , 0. , 0. , 0.1, 0.9]])},
 'result': {'samples': array([[ 0.86056952, -1.49661756,  2.31365488,  2.75722872,  4.44455054,
           3.72612187],
         [ 0.40330699, -0.82833824,  0.88654483,  2.83515146,  3.65749304,
           2.06392273],
         [ 1.5422168 ,  0.26487512,  1.52043224,  1.39434237,  2.73121381,
           3.85040991],
         ...,
         [-0.02026007,  2.19231154,  1.38474697,  2.40130853,  4.4686417 ,
           4.17862718],
         [-0.30469959,  2.03682192,  2.99402094,  2.95729883,  3.43736007,
           3.85773573],
         [ 0.42080381,  0.7